In [1]:
class Channel:
    def __init__(self, user,deposit):
        self.deposit = deposit
        self.user = user
        self.u2t = deposit/2
        self.t2u = deposit/2

In [2]:
class Transaction:
    def __init__(self, sender, receiver, amount, consent):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.consent = consent

In [3]:
class Tumbler:
    def __init__(self, balance):
        self.balance = balance
        self.channels = []
        
    def create_channel(self, user, desposit_amount):
        self.balance -= desposit_amount
        user.deposit(desposit_amount) 
        channel = Channel(user,desposit_amount*2)
        self.channels.append(channel)



In [4]:
class User:
    def __init__(self, user_id, balance):
        self.user_id = user_id
        self.balance = balance
        self.channel_balance = 0

    def deposit(self, amount):
        # Simulate depositing funds into the user's account
        self.balance -= amount
        self.channel_balance = amount
        return f"Deposited {amount} BTC into User {self.user_id}'s PCH account."
    
#     def initiate_tx(transaction):
        

    def withdraw(self, amount):
        # Simulate withdrawing funds from the user's account
        if amount > self.channel_balance:
            return "Insufficient funds."
        else:
            self.balance += amount
            self.channel_balance = 0
            return f"Withdrew {amount} BTC from User {self.user_id}'s PCH account."

### TumbleBit Implementation

In [5]:
import random


'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x, y = extended_gcd(b, a % b)
        return d, y, x - y * (a // b)

def find_multiplicative_inverse(a, m):
    d, x, y = extended_gcd(a, m)
    if d == 1:
        return (x % m + m) % m  # Ensure the result is positive and less than m
    else:
        raise ValueError("Multiplicative inverse does not exist.")


#returns True if a number is prime else False
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True




def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = find_multiplicative_inverse(e, phi)
    
#     print("KeyGen",e,n,d, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))




def encrypt(pk, plaintext):
    #Unpack the key into it's components
#     print(pk, plaintext)
    key, n = pk
#     print(key, n)
    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = (plaintext ** key) % n
    #Return the array of bytes
    return cipher




def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = (ciphertext ** key) % n
    #Return the array of bytes as a string
    return plain
    

In [6]:
def find_channel_by_userID(userID, channels):
    for channel in channels:
        if channel.user.user_id == userID:
            return channel
    return None

In [7]:
def transction_protocol(sender, receiver, tumbler, amount):
    print("\n\n")
    print("***************************************************************")

    print("Sender ID = "+str(sender.user_id))
    print("Receiver ID = "+str(receiver.user_id))
    
    sender_channel = find_channel_by_userID(sender.user_id, tumbler.channels)
    receiver_channel = find_channel_by_userID(receiver.user_id, tumbler.channels)
    
    if amount>sender_channel.u2t:
        print("Smart Contract: TX invalid, sender doesn't have enough balance!")
        return
    
    if amount>receiver_channel.t2u:
        print("Error! ET1: Receiver channel doesn't have enough balance!")
        return
    
    print("Information about channel between Sender and Tumbler:")
    print(vars(sender_channel))
    print("Information about channel between Tumbler and Receiver:")
    print(vars(receiver_channel))
    
    print("\n\n")
    print("***************************************************************")
    print("             Tumble Bit Transaction Protocol                   ")
    print("***************************************************************")

    print("TMB = Tumbler (a trustless 3rd party mediator)")
    print("\n\n--------------- Generating (z,c) at TMB ------------------------")
    temp=random.randrange(1000, 1500)
    while(is_prime(temp) != True):
        temp = random.randrange(1, 50)
    p = temp #taking any random no
    q = 197 #taking any random no

    #Generating your public/private keypairs
    public, private = generate_keypair(p, q)
    print("TMB (RSA public key) = ", public ," TMB (RSA private key) = ", private)
    
    while(amount):

        epsilon = random.randrange(1, 50)
        sigma = random.randrange(1, 50)
        print("Sigma = ",sigma,"epsilon = ",epsilon)
        z = decrypt(public, epsilon)
        c = sigma + epsilon


        print("\n\n\n")
        print("-------------- [TMB ---> Receiver Communication]-------------------")
        print("TMB sending (z,c) = (",z,", ",c,") to Receiver")




        print("\n\n\n")
        print("-------------- [Receiver ---> Sender Communication]-------------------")
        print("Blinding the z before sending")
        bf = random.randrange(1, 10)
        print("Blinding factor bf = ", bf,"& send zStar = z*(bf^p.k)")

        e,n = public
        zStar = (z*decrypt(public, bf))%n
        print("Receiver sending zStar Z* to Sender = ",zStar)
        print("verify zStar=",decrypt(public, bf*epsilon))




        print("\n\n\n")
        print("--------------[Sender ---> TMB Communication]-------------------")
        print("Sender request solution of zStar (z*) to TMB for 1 coin")


        epsilonStar=encrypt(private, zStar)
        print("epsilonStar = ",epsilonStar)
        x = random.randrange(1, 500)
        y = hash(x)
        q = epsilonStar + x



        print("Payment solver protocol exchange")
        for x in range(1,5):
            bff = random.randrange(1000, 5000)
            print("Blinding factors bf = ", bf)
        for x in range(1,5):
            temp = random.randrange(1000, 5000)
            print("Fake factors = ", temp)

        print("TMB sends (y,q) = (",y,q,") to Sender, with exchange for 1 coin.")
        sender_channel.u2t -= 1
        sender_channel.t2u += 1
        print("User with ID "+str(sender.user_id)+ " sent 1 coin to tumbler successfully")



        print("\n\n\n")
        print("--------------[Sender ---> Receiver Communication]-------------------")
        print("Sender sending epsilonStar to Receiver")
        print("Receiver calculating epsilon from epsilonStar")

        epsilonBob=epsilonStar / bf
        print("calculated epsilon at Receiver = ",epsilonBob)
        print("finding sigma from epsilon at Receiver")

        sigmaBob = c - epsilonBob
        print("sigma at Receiver = ",sigmaBob)



        print("\n\n\n")
        print("--------------[Receiver ---> TMB Communication]---------------------")
        print("Puzzle promise protocol exchange")
        for x in range(1,5):
            bf = random.randrange(1000, 5000)
            print("Blinding factor bf = ", bf," Hash = ",hash(bf))
        for x in range(1,5):
            temp = random.randrange(1000, 5000)
            print("Fake factors = ", temp)
        print("Receiver sending sigma = ",sigmaBob," to get 1 coin")

        receiver_channel.u2t += 1
        receiver_channel.t2u -= 1

        print("User with ID "+str(receiver.user_id)+ " recived 1 coin successfully")
        amount-=1

    
    print("Information about channel between Sender and Tumbler:")
    print(vars(sender_channel))
    print("Information about channel between Tumbler and Receiver:")
    print(vars(receiver_channel))

In [11]:
number_of_users=10

tumbler = Tumbler(balance=5000)

users = [User(user_id=i, balance=random.randint(50, 200)) for i in range(1, number_of_users+1)]

attacker =  User(user_id=number_of_users+1, balance=300)

users.append(attacker)


print("\nUsers:")
for i in users:
    print(vars(i))
    
print("\nTumbler Info:")
print(vars(tumbler))

# Users deposit funds
for user in users:
    if user.user_id<=number_of_users:
        desposit_amount=random.randint(10, 30)
        tumbler.create_channel(user, desposit_amount)
    else:
        desposit_amount = int(0.8*attacker.balance)
        tumbler.create_channel(attacker, desposit_amount)
        

print("\nUsers (after channel formation):")
for i in users:
    print(vars(i))

    
print("\nTumbler Info (after channel formation):")
print(vars(tumbler))

print("\nChannels with the Tumbler:")
for i in tumbler.channels:
    print(vars(i))

num_of_txs = 50

transction_protocol(sender=users[0], receiver=users[1], tumbler=tumbler, amount = 5)


Users:
{'user_id': 1, 'balance': 94, 'channel_balance': 0}
{'user_id': 2, 'balance': 55, 'channel_balance': 0}
{'user_id': 3, 'balance': 162, 'channel_balance': 0}
{'user_id': 4, 'balance': 75, 'channel_balance': 0}
{'user_id': 5, 'balance': 165, 'channel_balance': 0}
{'user_id': 6, 'balance': 148, 'channel_balance': 0}
{'user_id': 7, 'balance': 144, 'channel_balance': 0}
{'user_id': 8, 'balance': 126, 'channel_balance': 0}
{'user_id': 9, 'balance': 148, 'channel_balance': 0}
{'user_id': 10, 'balance': 176, 'channel_balance': 0}
{'user_id': 11, 'balance': 300, 'channel_balance': 0}

Tumbler Info:
{'balance': 5000, 'channels': []}

Users (after channel formation):
{'user_id': 1, 'balance': 80, 'channel_balance': 14}
{'user_id': 2, 'balance': 43, 'channel_balance': 12}
{'user_id': 3, 'balance': 141, 'channel_balance': 21}
{'user_id': 4, 'balance': 49, 'channel_balance': 26}
{'user_id': 5, 'balance': 143, 'channel_balance': 22}
{'user_id': 6, 'balance': 133, 'channel_balance': 15}
{'user